# BIBLIOTECAS

In [2]:
!pip3 install pandas numpy seaborn matplotlib

In [3]:
import pandas as pd
import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import glob

# SANITY CHECK

## Objetivos:

- Entendimento inicial de cada uma das bases de dados
- Analisar propriedades das colunas das bases
- Identificar e tratar possíveis inconsistências nos dados

In [13]:
# Obter a lista de arquivos CSV no diretório
csv_files = glob.glob('../data/raw/*.csv')

df_list = [pd.read_csv(file) for file in csv_files]

In [17]:
csv_files

['../data/raw\\DemographicData_ZCTAs.csv',
 '../data/raw\\df_geocode (1).csv',
 '../data/raw\\EconomicData_ZCTAs (1).csv',
 '../data/raw\\exams_data.csv',
 '../data/raw\\transactional_data (1).csv']

In [ ]:
demographic_df = pd.read_csv(csv_files[0])
geocode_df = pd.read_csv(csv_files[1])
economic_df = pd.read_csv(csv_files[2])
exams_df = pd.read_csv(csv_files[3])
transactional_df = pd.read_csv(csv_files[4])

Dataframes criados: demographic_df, geocode_df, economic_df, exams_df, transactional_df


Dataframes criados:
- demographic_df
- geocode_df
- economic_df
- exams_df
- transactional_df

### Visualização do demographic_df

In [38]:
print("Colunas do dataframe demographic:", demographic_df.columns)
print("Shape do dataframe demographic", demographic_df.shape)
print(demographic_df.info())

Colunas do dataframe demographic: Index(['Unnamed: 0', 'Id', 'GeographicAreaName', 'TotalPopulation', 'SexRatio(males per 100 females)', 'Population_Under5Years', 'Population_5to9Years', 'Population_10to14Years', 'Population_15to19Years', 'Population_20to24Years', 'Population_25to34Years', 'Population_35to44Years', 'Population_45to54Years', 'Population_55to59Years', 'Population_60to64Years', 'Population_65to74Years', 'Population_75to84Years', 'Population_85YearsAndOver', 'MedianAgeInYears'], dtype='object')
Shape do dataframe demographic (33120, 19)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33120 entries, 0 to 33119
Data columns (total 19 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Unnamed: 0                       33120 non-null  int64  
 1   Id                               33120 non-null  object 
 2   GeographicAreaName               33120 non-null  object 
 3   TotalPopulation     

In [ ]:
pd.set_option('display.width', 600)
pd.set_option('display.max_columns', None)
print(demographic_df.head(5))

   Unnamed: 0              Id GeographicAreaName  TotalPopulation  SexRatio(males per 100 females)  Population_Under5Years  Population_5to9Years  Population_10to14Years  Population_15to19Years  Population_20to24Years  Population_25to34Years  Population_35to44Years  Population_45to54Years  Population_55to59Years  Population_60to64Years  Population_65to74Years  Population_75to84Years  Population_85YearsAndOver  MedianAgeInYears
0           0  8600000US35004        ZCTA5 35004            12045                             94.1                     805                  1075                     898                     477                     578                    2088                    1628                    1200                     886                     683                    1017                     534                        176              35.5
1           1  8600000US35005        ZCTA5 35005             7344                             86.1                     504                  

In [62]:
duplicated_rows = demographic_df[demographic_df.duplicated(subset=['Id', 'GeographicAreaName'])]
print(duplicated_rows)

Empty DataFrame
Columns: [Unnamed: 0, Id, GeographicAreaName, TotalPopulation, SexRatio(males per 100 females), Population_Under5Years, Population_5to9Years, Population_10to14Years, Population_15to19Years, Population_20to24Years, Population_25to34Years, Population_35to44Years, Population_45to54Years, Population_55to59Years, Population_60to64Years, Population_65to74Years, Population_75to84Years, Population_85YearsAndOver, MedianAgeInYears]
Index: []


In [63]:
print(f"Quantidade de valores únicos na coluna 'Id': {demographic_df['Id'].nunique()}")
print(f"Quantidade de valores únicos na coluna 'GeographicAreaName': {demographic_df['GeographicAreaName'].nunique()}")

Quantidade de valores únicos na coluna 'Id': 33120
Quantidade de valores únicos na coluna 'GeographicAreaName': 33120


Esses valores correspondem ao número total de linhas

Nota-se que a maioria das colunas da base é numérica, com características relacionadas à demografia de áreas geográficas. A maior parte das variáveis que descrevem as características populacionais são numéricas, porém existem variáveis aparentemente categóricas:

- `Id` (Identificador único da área geográfica)
- `GeographicAreaName` (Nome da área geográfica)
- `SexRatio(males per 100 females)` (Razão de sexo: número de homens por 100 mulheres)


### Visualização do geocode_df

In [39]:
print("Colunas do dataframe geocode:", geocode_df.columns)
print("Shape do dataframe geocode", geocode_df.shape)
print(geocode_df.info())

Colunas do dataframe geocode: Index(['Unnamed: 0', 'Lab Id', 'Address', 'Location', 'Zipcode'], dtype='object')
Shape do dataframe geocode (119, 5)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119 entries, 0 to 118
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  119 non-null    int64  
 1   Lab Id      119 non-null    object 
 2   Address     119 non-null    object 
 3   Location    119 non-null    object 
 4   Zipcode     118 non-null    float64
dtypes: float64(1), int64(1), object(3)
memory usage: 4.8+ KB
None


In [41]:
print(geocode_df.head(5))

   Unnamed: 0 Lab Id                                            Address                 Location  Zipcode
0           0   L152      3800 PLEASANT HILL RD STE 1, DULUTH, GA 30096    34.000319,-84.1629724  30096.0
1           1   L520                    1614 N JAMES ST, ROME, NY 13440   43.2311327,-75.4445363  13440.0
2           2   L141     12911 120TH AVE NE STE D60, KIRKLAND, WA 98034  47.7162786,-122.1838152  98034.0
3           3   L524  5667 PEACHTREE DUNWOODY RD 250, ATLANTA, GA 30342   33.9093875,-84.3529096  30342.0
4           4   L545                 1204 IL HWY 164, OQUAWKA, IL 61469   40.9309925,-90.9437598  61469.0


### Visualização do economic_df

In [42]:
print("Colunas do dataframe economic:", economic_df.columns)
print("Shape do dataframe economic", economic_df.shape)
print(economic_df.info())

Colunas do dataframe economic: Index(['Unnamed: 0', 'id', 'Geographic Area Name', 'TotalHouseholds_LessThan$10.000', 'TotalHouseholds_$10.000to$14.999', 'TotalHouseholds_$15.000to$24.999', 'TotalHouseholds_$25.000to$34.999', 'TotalHouseholds_$35.000to$49.999', 'TotalHouseholds_$50.000to$74.999', 'TotalHouseholds_$75.000to$99.999', 'TotalHouseholds_$100.000to$149.999', 'TotalHouseholds_$150.000to$199.999', 'TotalHouseholds_$200.000OrMore'], dtype='object')
Shape do dataframe economic (629280, 13)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 629280 entries, 0 to 629279
Data columns (total 13 columns):
 #   Column                              Non-Null Count   Dtype 
---  ------                              --------------   ----- 
 0   Unnamed: 0                          629280 non-null  int64 
 1   id                                  629280 non-null  object
 2   Geographic Area Name                629280 non-null  object
 3   TotalHouseholds_LessThan$10.000     629280 non-null  int64

In [43]:
pd.set_option('display.width', 600)
pd.set_option('display.max_columns', None)
print(economic_df.head(5))

   Unnamed: 0              id Geographic Area Name  TotalHouseholds_LessThan$10.000  TotalHouseholds_$10.000to$14.999  TotalHouseholds_$15.000to$24.999  TotalHouseholds_$25.000to$34.999  TotalHouseholds_$35.000to$49.999  TotalHouseholds_$50.000to$74.999  TotalHouseholds_$75.000to$99.999  TotalHouseholds_$100.000to$149.999  TotalHouseholds_$150.000to$199.999  TotalHouseholds_$200.000OrMore
0           0  8600000US35004          ZCTA5 35004                              198                                71                               298                               513                               647                              1117                               529                                 945                                 245                              61
1           1  8600000US35005          ZCTA5 35005                              188                               184                               318                               293                               

### Visualização do exams_df

In [44]:
print("Colunas do dataframe exams:", exams_df.columns)
print("Shape do dataframe exams", exams_df.shape)
print(exams_df.info())

Colunas do dataframe exams: Index(['CodItem', 'Desc Item', 'Category', 'Family', 'Speciality', 'Testing Cost'], dtype='object')
Shape do dataframe exams (2002, 6)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2002 entries, 0 to 2001
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   CodItem       2002 non-null   int64  
 1   Desc Item     2002 non-null   object 
 2   Category      2002 non-null   object 
 3   Family        2002 non-null   object 
 4   Speciality    2002 non-null   object 
 5   Testing Cost  2002 non-null   float64
dtypes: float64(1), int64(1), object(4)
memory usage: 94.0+ KB
None


In [52]:
print(exams_df.isnull().sum())

CodItem         0
Desc Item       0
Category        0
Family          0
Speciality      0
Testing Cost    0
dtype: int64


In [ ]:
print(exams_df.head(5))

    CodItem                                          Desc Item Category   Family     Speciality  Testing Cost
0  70003237  MAGNESIO, SORO (EXEMPLO DE EXPRESSAO DE RESULT...       CA  CORELAB   BIOCHEMISTRY          1.78
1  70000638                    HEMOGRAMA, AUTOMATIZADO, SANGUE       CA   HEMATO    BLOOD COUNT          2.46
2  70001597                                    FERRITINA, SORO       CA  CORELAB  IMMUNOHORMONE          2.11
3  70000103  FERRO, DOSAGEM, SORO (EXEMPLO DE EXPRESSAO DE ...       CA  CORELAB   BIOCHEMISTRY          0.80
4  70000224  CALCIO, DOSAGEM, SORO (EXEMPLO DE EXPRESSAO DE...       CA  CORELAB   BIOCHEMISTRY          1.02


### Visualização do transactional_df

In [ ]:
print("Colunas do dataframe transactional:", transactional_df.columns)
print("Shape do dataframe transactional", transactional_df.shape)
print(transactional_df.info())

Colunas do dataframe transactional: Index(['Patient Id;Gender;Date of birth;Date of service;Service Id;Lab Id;CodItem;Testing Cost'], dtype='object')
Shape do dataframe transactional (2355241, 1)
<class 'pandas.core.frame.DataFrame'>
Index: 2355241 entries, 10210830256-BIO003;F;01/08/1976 00:00:00;2019-01-07;571904533475-38;L133;70003237;9 to 7664157546-1;M;06/03/1971 00:00:00;2021-02-12;7664157546-1-1;L697;70000392;8
Data columns (total 1 columns):
 #   Column                                                                                  Dtype
---  ------                                                                                  -----
 0   Patient Id;Gender;Date of birth;Date of service;Service Id;Lab Id;CodItem;Testing Cost  int64
dtypes: int64(1)
memory usage: 35.9+ MB
None


Aqui observamos que o dataframe foi lido de maneira incorreta, então precisou-se ajustar para fazer a leitura correta

In [ ]:
transactional_df = pd.read_csv(csv_files[4], sep=';', index_col=False)

print("Colunas do dataframe transactional ajustado:", transactional_df.columns)
print("Shape do dataframe transactional ajustado", transactional_df.shape)
print(transactional_df.info())

Colunas do dataframe transactional ajustado: Index(['Patient Id', 'Gender', 'Date of birth', 'Date of service', 'Service Id', 'Lab Id', 'CodItem', 'Testing Cost'], dtype='object')
Shape do dataframe transactional ajustado (2355241, 8)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2355241 entries, 0 to 2355240
Data columns (total 8 columns):
 #   Column           Dtype 
---  ------           ----- 
 0   Patient Id       object
 1   Gender           object
 2   Date of birth    object
 3   Date of service  object
 4   Service Id       object
 5   Lab Id           object
 6   CodItem          int64 
 7   Testing Cost     object
dtypes: int64(1), object(7)
memory usage: 143.8+ MB
None


In [60]:
print(transactional_df.isnull().sum())

Patient Id         0
Gender             0
Date of birth      1
Date of service    0
Service Id         0
Lab Id             0
CodItem            0
Testing Cost       0
dtype: int64


In [61]:
print(transactional_df[transactional_df.isnull().any(axis=1)])

           Patient Id Gender Date of birth Date of service      Service Id Lab Id   CodItem  Testing Cost
1253947  7659985004-1      F           NaN      2020-02-22  7659985004-2-1   L152  70000107          48.0


Preciso entender como tratar essa linha

In [ ]:
# Remover espaços em branco e normalizar a coluna 'Testing Cost'
transactional_df['Testing Cost'] = transactional_df['Testing Cost'].str.replace(',', '.').astype(float)

# Verificar duplicatas considerando todas as colunas
duplicated_rows = transactional_df[transactional_df.duplicated()]
print(duplicated_rows.head(2))

             Patient Id Gender        Date of birth Date of service        Service Id Lab Id   CodItem  Testing Cost
110  10240091057-BIO010      F  12/07/1976 00:00:00      2019-01-09  430000507035-168   L536  70005005          59.0
325  10240100137-BIO010      F  19/01/1987 00:00:00      2019-02-03  431300349554-172   L539  70000180          12.0


In [56]:
transactional_df[transactional_df.duplicated()]

,Patient Id,Gender,Date of birth,Date of service,Service Id,Lab Id,CodItem,Testing Cost
110,10240091057-BIO010,F,12/07/1976 00:00:00,2019-01-09,430000507035-168,L536,70005005,"59,0"
325,10240100137-BIO010,F,19/01/1987 00:00:00,2019-02-03,431300349554-172,L539,70000180,"12,0"
326,10240100137-BIO010,F,19/01/1987 00:00:00,2019-02-03,431300349554-172,L539,70000180,"12,0"
731,10210824046-BIO003,F,13/04/1957 00:00:00,2019-01-15,576401751254-89,L166,70000180,"12,0"
732,10210824046-BIO003,F,13/04/1957 00:00:00,2019-01-15,576401751254-89,L166,70000180,"12,0"
...,...,...,...,...,...,...,...,...
2354236,6004062388-1,F,13/09/1986 00:00:00,2021-01-13,6004062388-3-1,L139,70000819,"30,0"
2354238,6004062388-1,F,13/09/1986 00:00:00,2021-01-13,6004062388-3-1,L139,70006315,"26,0"
2354554,7664121334-1,F,07/11/1947 00:00:00,2021-02-09,7664121334-2-1,L155,70002747,"81,0"
2354885,7658735373-1,M,08/11/1978 00:00:00,2021-01-29,7658735373-15-1,L158,70000180,"12,0"


In [50]:
pd.set_option('display.width', 600)
pd.set_option('display.max_columns', None)
print(transactional_df.head(5))

           Patient Id Gender        Date of birth Date of service       Service Id Lab Id   CodItem Testing Cost
0  10210830256-BIO003      F  01/08/1976 00:00:00      2019-01-07  571904533475-38   L133  70003237          9,0
1  10210830256-BIO003      F  01/08/1976 00:00:00      2019-01-07  571904533475-38   L133  70000638         13,0
2  10210830256-BIO003      F  01/08/1976 00:00:00      2019-01-07  571904533475-38   L133  70001597         49,0
3  10210830256-BIO003      F  01/08/1976 00:00:00      2019-01-07  571904533475-38   L133  70000103         11,0
4  10210830256-BIO003      F  01/08/1976 00:00:00      2019-01-07  571904533475-38   L133  70000224         10,0


# EDA